In [ ]:
import numpy as np
import graphinglib as gl
from astropy.io import fits
from astropy.constants import c as light_speed
from matplotlib.colors import ListedColormap

from src.hdu.map import Map
from src.hdu.cube import Cube
from src.hdu.header import Header
from src.hdu.grouped_maps import GroupedMaps
from src.coordinates.celestial_coords import RA, DEC
from src.coordinates.fits_coords import FitsCoords
from src.tools.loki import get_loki_line_pdfs_figure, get_loki_fit_figure


gm = GroupedMaps.load_from_loki(
    "data/loki/output_NGC4696_G235H_F170LP_full_OQBr_tied/NGC4696_G235H_F170LP_full_OQBr_tied_parameter_maps.fits"
)
SNR_cut = 3

# Finding the AGN pixel coordinate
# --------------------------------
header = gm["LINES.H210_Q7.TOTAL_FLUX"].header
world_coords = [RA.from_sexagesimal("12:48:49.2609").degrees, DEC.from_sexagesimal("-41:18:39.417").degrees]
coords = header.world_to_pixel(world_coords)[0][::-1]  # x, y
print(coords)

# Rotate this coord
# -----------------
# theta = np.radians(-47)
# rotated_coords = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]]) @ coords
# print(rotated_coords)

# GL useful elements
arrows = [
    gl.Arrow((7.8+20, 2-23), (12.8+20, 7-23), "black"),
    gl.Arrow((8.2+20, 2-23), (3.2+20, 7-23), "black"),
    gl.Text(13+20, 7-23, "N", color="black", font_size=15, h_align="center", v_align="bottom"),
    gl.Text(3+20, 7-23, "E", color="black", font_size=15, h_align="center", v_align="bottom"),
]
AGN_pos = gl.Point(40.3, 0, marker_style="x", color="red", marker_size=50)

In [ ]:
v4 = fits.open("f170lp_g235h-f170lp_s3d.fits")
v4a = fits.open("f170lp_g235h-f170lp_s3d (1).fits")
v4b = fits.open("f170lp_g235h-f170lp_s3d_4b.fits")
v4.info()
v4_cube = Cube(v4[1].data, v4[1].header)
v4b_cube = Cube(v4b[1].data, v4b[1].header)

print(v4_cube[2130, 80, 29])
print(v4b_cube[2130, 80, 29])

print(v4_cube == v4b_cube)

### Fig 1

In [ ]:
data = fits.open(
    "data/output_NGC4696_G235H_F170LP_full_model/NGC4696_G235H_F170LP_full_model_parameter_maps.fits"
)[67].data  # H_2(1-0)
hm = gl.Heatmap(data, origin_position="lower", show_color_bar=False, color_map_range=(-18, -16))
zoomed_hm = gl.Heatmap(data[24:33,24:33], origin_position="lower", show_color_bar=False, color_map_range=(-18, -16))
zoomed_hm_2 = zoomed_hm.copy()
zoomed_hm_2.image = zoomed_hm.image[2:7,2:7]
zoomed_hm_2.color_map = "plasma"

fig = gl.SmartFigure(
    2,
    2,
    remove_x_ticks=True,
    remove_y_ticks=True,
    size=(11, 6),
    width_ratios=[5, 2],
    reference_labels=False,
    width_padding=0,
    height_padding=0,
).set_visual_params(use_latex=True)
fig[:, 0] = [
    hm,
    gl.Text(1, 54, "NGC 4696 ($z=0.0104$)", h_align="left"),
    gl.Text(1, 52, "Central dominant galaxy in the Centaurus Cluster", h_align="left"),
    gl.Text(1, 3, r"H$\alpha$ HST ...", h_align="left"),
    gl.Text(1, 1, r"Chandra X-ray (blue) ...", h_align="left"),
    gl.Text(28, 35, r"[Chandra X-ray + H$\alpha$ HST image"+"\nof the whole cluster]", font_size=20),
    gl.Line((45, 1), (55, 1), capped_line=True, cap_width=0.5, width=1),
    gl.Text(50, 2, "x kpc"),
    gl.Rectangle(23.5, 23.5, 9, 9, fill=False),
    gl.Polygon([[9+23.5, 23.5], [9+23.5, 9+23.5], [60, 60], [60, 28.7]], fill=False),
]
fig[0, 1] = [
    zoomed_hm,
    gl.Text(4, 3, "[Identify with\narrows the AGN\nand filaments]", font_size=15),
    gl.Text(6.5, 7.3, "x kpc"),
    gl.Line((5, 7), (8, 7), capped_line=True, cap_width=0.5, width=1),
    gl.Rectangle(1.5, 1.5, 5, 5, fill=False),
    gl.Polygon([[-0.5, -0.5], [1.5, 1.5], [6.5, 1.5], [9.5, -1.5]], fill=False),
]
fig[1, 1] = [
    zoomed_hm_2,
    gl.Rectangle(0.5, 0.5, 3, 3, fill=False),
    gl.Text(2, 4, "[MUSE velocity field]", font_size=15),
    gl.Text(2, 3.1, "[NIRSpec's FOV]", font_size=15),
    gl.Line((2, 0), (4, 0), capped_line=True, cap_width=0.5, width=1),
    gl.Text(3, 0.2, "x kpc"),
]
fig.show().save("figures/article/fig_1.pdf", dpi=600)

### Fig 2

In [ ]:
f = fits.open("data/radio/PKS1246-410.fits")
vlba_map = Map(f[0].data[0, 0], header=Header(f[0].header).celestial)#.get_reprojection_on(header)
s1_map = gm["LINES.H210_S1.TOTAL_FLUX"].get_reprojection_on(vlba_map.header)

x_mesh, y_mesh = np.meshgrid(np.linspace(0, s1_map.shape[0], vlba_map.data.shape[0]), np.linspace(0, s1_map.shape[1], vlba_map.data.shape[1]))
# gl.SmartFigure(elements=[gl.Contour(x_mesh, y_mesh, vlba_map.data, 5, filled=False, color_map=ListedColormap("red"), color_map_range=(0.002, 0.01), show_color_bar=True)]).show()
# gl.SmartFigure(elements=[s1_map.data.plot, gl.Contour(x_mesh, y_mesh, vlba_map.data, 5, filled=False, color_map=ListedColormap("red"), color_map_range=(0.002, 0.01), show_color_bar=True)]).show()

from reproject import reproject_interp

reproj_data, footprint = reproject_interp(
    (s1_map.data, s1_map.header),
    vlba_map.header,
    order="bilinear"   # or "bicubic" for smoother sub-pixel features
)
gl.SmartFigure(elements=[gl.Heatmap(reproj_data)]).show()

In [ ]:
lines = [f"LINES.H210_S1.{p}" for p in ["TOTAL_FLUX", "VPEAK", "1.FWHM"]]
lines += [f"CONTINUUM.STELLAR_{p}" for p in ["KINEMATICS.VEL", "KINEMATICS.VDISP"]]
hms = []
texts = [gl.Text(59, 19.5, "H$_2$ 1-0 $S(1)$", h_align="right", v_align="top")]*3

for i, line in zip([0, 1, 2, 4, 5], lines):
    map_ = gm[line]
    # map_.data[*np.round(coords).astype(int)] = np.nan
    if i < 3:
        # Mask with an SNR cut
        map_ = map_.mask(gm[f"{line[:14]}TOTAL_SNR"].data > SNR_cut)

    match i % 3:
        case 0: cmap = "plasma"
        case 1: cmap = "coolwarm"
        case 2: cmap = "viridis"
    match i:
        case 0: cbar_label = r"$\log_{10}(F_\mathrm{gas}/$ erg s$^{-1}$ cm$^{-2})$"
        case 1: cbar_label = r"velocity $v_\mathrm{gas}$ (km s$^{-1}$)"
        case 2:
            map_ /= 2 * np.sqrt(2 * np.log(2))  # FWHM to sigma
            cbar_label = r"velocity dispersion $\sigma_\mathrm{gas}$ (km s$^{-1}$)"
        # case 3: cbar_label = r"$\log_{10}(M_\star/M_{\odot})$"
        case 4: cbar_label = r"velocity $v_\star$ (km s$^{-1}$)"
        case 5: cbar_label = r"velocity dispersion $\sigma_\star$ (km s$^{-1}$)"

    hm = map_.rotate_field()
    hm.color_map = cmap
    if i < 3: hm.set_color_bar_params(position="top", label=cbar_label)
    else: hm.set_color_bar_params(position="bottom", label=cbar_label)
    hms.append(hm)
for i, cmap_range in enumerate([(-18.27, -17.53), (-460, 250), (30, 340), (-50, 50), (230, 325)]):
    hms[i].color_map_range = cmap_range

AGN_pos = gl.Point(40.3, 0, marker_style="x", color="red", marker_size=50)
# AGN_text = gl.Text(37.3, 2, r"$\textbf{AGN}$", color="red", font_size=14, h_align="center", v_align="center")

print(hms[0]._x_coordinates.shape)
print(hms[0]._y_coordinates.shape)
print(hms[0].image.shape)

# Radio contours
f = fits.open("data/radio/PKS1246-410.fits")
# vlba_map = Map(f[0].data[0, 0], header=Header(f[0].header).celestial).get_reprojection_on(header)
# radio_contours = gl.Contour(
#     hms[0]._x_coordinates,
#     hms[0]._y_coordinates,
#     vlba_map.data,
#     5,
#     ListedColormap(["red"]),
#     show_color_bar=False,
#     filled=False,
#     alpha=0.5,
# )
radio_contours = None

# radio_contours = gl.Contour.from_function(
#     lambda x, y: np.exp(-0.5*(((x-40.3)/6)**2 + ((y-0)/3)**2)), (20, 60), (-23, 20), 2, ListedColormap(["red"]),
#     show_color_bar=False, filled=False, alpha=0.5,
# )

fig = gl.SmartFigure(
    2,
    3,
    # aspect_ratio=1,
    size=(9.12, 8),
    remove_x_ticks=True,
    remove_y_ticks=True,
    elements=hms[:3] + [None] + hms[3:],
    reference_labels_loc=(0.03, -0.16),
    width_padding=0,
    height_padding=0,
    x_lim=(20, 60),
    y_lim=(-23, 20),
)
fig.add_elements(*texts)
fig.add_elements(arrows)
for i in [0, 1, 2, 4, 5]:
    fig[i // 3, i % 3] += [AGN_pos, radio_contours]

fig.set_visual_params(use_latex=True).show().save("figures/paper_1/results/stellar_kinematics/fig_2_v5.pdf", dpi=600)

### Fig 3

In [ ]:
# First page of Fig.3 will contain the S1 -> S6 maps
lines = [f"H210_S{i}" for i in range(1, 7)]
# get_loki_line_pdfs_figure(lines).save("figures/tests/all_S.pdf", dpi=600)
texts = []
for i in range(1, 7):
    texts.extend([gl.Text(48, 16, rf"H$_2$ 1-0 $S({i})$", font_size=10)]*3)
    # texts.extend([rf"H$_2$ $1-0$ $S({i}$)", None, None])

hms = []
for i, line in enumerate(lines):
    for type_ in ["TOTAL_FLUX", "VPEAK", "1.FWHM"]:
        map_ = gm[f"LINES.{line}.{type_}"]
        map_ = map_.mask(gm[f"LINES.{line}.TOTAL_SNR"].data > SNR_cut)
        hms.append(map_.rotate_field())

    hms[-3].color_map = "plasma"
    hms[-2].color_map = "coolwarm"
    hms[-1].color_map = "viridis"
    hms[-1].image /= 2 * np.sqrt(2 * np.log(2))  # FWHM to sigma
    hms[-3].color_map_range = -18.4, -17.2
    hms[-2].color_map_range = -450, 450
    hms[-1].color_map_range = 50, 340

for hm in hms[3:]:
    hm.show_color_bar = False

hms[0].set_color_bar_params(position="top", label=r"$\log(F/$ erg s$^{-1}$ cm$^{-2})$")
hms[1].set_color_bar_params(position="top", label="$v$ (km s$^{-1}$)")
hms[2].set_color_bar_params(position="top", label=r"$\sigma$ (km s$^{-1}$)")

fig = gl.SmartFigure(
    6,
    3,
    # aspect_ratio=1,
    size=(4.85, 11),
    remove_x_ticks=True,
    remove_y_ticks=True,
    elements=hms,
    reference_labels_loc=(0.03, -0.16),
    width_padding=0,
    height_padding=0,
    x_lim=(20, 60),
    y_lim=(-23, 20),
    height_ratios=[1.33, 1, 1, 1, 1, 1],
    reference_labels=False,
).set_visual_params(use_latex=True)
fig.add_elements(*[arrows]*3)
fig.add_elements(*[AGN_pos]*18)
fig.add_elements(*texts)
# fig.show()
fig.save("figures/article/fig_3_1_v2.pdf", dpi=600)

In [ ]:
# Second page of Fig.3 will contain the O2, Br beta, O3, Pa alpha, Q7 maps
# lines = ["H210_O2", "H210_O3", "H210_Q7", "HI_Br_beta", "HI_Pa_alpha"]
# get_loki_line_pdfs_figure(lines).save("figures/tests/all_fig_3_2.pdf", dpi=600)
lines = ["H210_O2", "H210_O3", "H210_Q7", "HI_BR_BETA", "HI_PA_ALPHA"]
texts = []
for label in ["H$_2$ 1-0 $O(2)$", "H$_2$ 1-0 $O(3)$", "H$_2$ 1-0 $Q(7)$", r"Br$\beta$", r"Pa$\alpha$"]:
    texts.extend([gl.Text(58, 16, label, font_size=10, h_align="right")]*3)

hms = []
for i, line in enumerate(lines):
    for type_ in ["TOTAL_FLUX", "VPEAK", "1.FWHM"]:
        map_ = gm[f"LINES.{line}.{type_}"]
        map_ = map_.mask(gm[f"LINES.{line}.TOTAL_SNR"].data > SNR_cut)
        hms.append(map_.rotate_field())

    hms[-3].color_map = "plasma"
    hms[-2].color_map = "coolwarm"
    hms[-1].color_map = "viridis"
    hms[-1].image = hms[-1].image / (2 * np.sqrt(2 * np.log(2)))  # FWHM to sigma
    hms[-3].color_map_range = -18.8, -17.2
    hms[-2].color_map_range = -450, 450
    hms[-1].color_map_range = 50, 340

for hm in hms[3:]:
    hm.show_color_bar = False

hms[0].set_color_bar_params(position="top", label=r"$\log(F/$ erg s$^{-1}$ cm$^{-2})$")
hms[1].set_color_bar_params(position="top", label="$v$ (km s$^{-1}$)")
hms[2].set_color_bar_params(position="top", label=r"$\sigma$ (km s$^{-1}$)")

fig = gl.SmartFigure(
    5,
    3,
    # aspect_ratio=1,
    size=(5.8, 11),
    remove_x_ticks=True,
    remove_y_ticks=True,
    elements=hms,
    reference_labels_loc=(0.03, -0.16),
    width_padding=0,
    height_padding=0,
    x_lim=(20, 60),
    y_lim=(-23, 20),
    height_ratios=[1.285, 1, 1, 1, 1],
    reference_labels=False,
).set_visual_params(use_latex=True)
fig.add_elements(*[arrows]*3)
fig.add_elements(*[AGN_pos]*15)
fig.add_elements(*texts)
# fig.show()
fig.save("figures/article/fig_3_2_v2.pdf", dpi=600)

### Tests

In [ ]:
file = "output_NGC4696_G235H_F170LP_full_model_stars_full"
# file = "output_NGC4696_G235H_F170LP_full_model_stars_Pa+H2"

lines = [f"H210_{i}" for i in ["O2", "O3", "O4", "Q6", "Q7", "Q8", "Q9", "S0", "S1", "S2", "S3", "S4", "S5", "S6", "S7", "S8"]]
lines = ["HI_Pa_alpha", "H210_S3"]

fig = get_loki_line_pdfs_figure(lines, file)
fig.save("figures/workshop/test_1.pdf", dpi=600)

In [ ]:
file = "output_NGC4696_G235H_F170LP_full_OQBr_tied"

lines = ["H210_S1", "H210_S3", "H210_S5", "HI_Pa_alpha", "H210_O3"]

fig = get_loki_line_pdfs_figure(lines, file)
fig.save("figures/workshop/newest_lines.pdf", dpi=600)

In [ ]:
from src.hdu.cube import Cube
import graphinglib as gl

cube = Cube.load("data/f170lp_g235h-f170lp_s3d_choiclip1_2_wicked.fits", 1)
df = cube.get_deep_frame()
gl.SmartFigure(elements=[df.data.plot]).show()

In [ ]:
# October results
get_loki_fit_figure(
    "data/loki/output_NGC4696_G235H_F170LP_full_model_stars_full/NGC4696_G235H_F170LP_full_model_stars_full_full_model.fits",
    (32, 27),
).show()

In [ ]:
# December results
%matplotlib tk
get_loki_fit_figure(
    "data/loki/output_NGC4696_G235H_F170LP_full_OQBr_tied/NGC4696_G235H_F170LP_full_OQBr_tied_full_model.fits",
    FitsCoords(45, 37),
    version=3,
).show()

In [ ]:
# December individual spaxels
get_loki_fit_figure(
    "data/loki/32_27/output_NGC4696_G235H_F170LP_32_27_QOBr_tied/NGC4696_G235H_F170LP_32_27_QOBr_tied_full_model.fits",
    FitsCoords(1, 1),
    version=3,
).show()

In [ ]:
# December individual spaxels
fig1 = get_loki_fit_figure(
    "data/loki/32_27/output_NGC4696_G235H_F170LP_32_27_allH2Br_tied/NGC4696_G235H_F170LP_32_27_allH2Br_tied_full_model.fits",
    FitsCoords(1, 1),
    version=3,
)
fig2 = get_loki_fit_figure(
    "data/loki/32_37/output_NGC4696_G235H_F170LP_32_37_allH2Br_tied/NGC4696_G235H_F170LP_32_37_allH2Br_tied_full_model.fits",
    FitsCoords(1, 1),
    version=3,
)
other_data_curve = fig2[1][0]
other_data_curve.color = "lime"
other_data_curve.line_width = 0.5
fig1[1] += [other_data_curve]
%matplotlib tk
fig1.show()

In [ ]:
voff = gm["LINES.H210_S4.1.VOFF"]
delta_v = gm["LINES.H210_S4.DELTA_V"]
voff_hm = voff.data.plot
delta_v_hm = delta_v.data.plot
voff_hm.color_map_range = -600, 600
delta_v_hm.color_map_range = -600, 600
gl.SmartFigure(1, 2, elements=[voff_hm, delta_v_hm]).show()

In [ ]:
f_wicked = fits.open("data/f170lp_g235h-f170lp_s3d_choiclip1_2_wicked.fits")
gl.SmartFigure(
    elements=[
        gl.Curve(np.arange(f_wicked[3].data.shape[0]), f_wicked[3].data[:,*FitsCoords(3, 29)]),
        gl.Curve(np.arange(f_wicked[1].data.shape[0]), f_wicked[1].data[:,*FitsCoords(3, 29)])
    ],
    x_label="Channel",
    y_label="DQ value",
    title="DQ array for spaxel (3, 29)",
).show()#.save("test_dq.pdf")

In [ ]:
f_wicked = fits.open("data/wicked/f170lp_g235h-f170lp_s3d_choiclip1_2_wicked.fits")
f_loki = fits.open("data/loki/output_NGC4696_G235H_F170LP_full_OQBr_tied/NGC4696_G235H_F170LP_full_OQBr_tied_full_model.fits")
# print(f_wicked[1].data.shape[0], f_loki[1].data.shape[0])
# raise

# f_wicked.info()
# print(f_wicked[1].header["CDELT3"], f_wicked[1].header["NAXIS3"])
# print(f_wicked[5].data)
# raise

wave = f_wicked[1].header["CRVAL3"] + f_wicked[1].header["CDELT3"] * np.arange(0, f_wicked[1].header["NAXIS3"])

wv = f_loki[-1].data[0][0].flatten()
# print(wv)
# ∆lambda / lambda:
delta_lambda_over_lambda = np.diff(wv) / wv[:-1]
print(delta_lambda_over_lambda)

gl.SmartFigure(elements=[
        gl.Curve(wave, f_wicked[1].data[:, 32, 27], label="WICKED"),
        gl.Curve(wv, f_loki[1].data[:, 32, 27] * 1e17, label="LOKI"),
    ],
    # x_lim=(1.66, 1.75),
    x_lim=(3.05, 3.1),
    # y_lim=(70, 90),
    y_lim=(34, 40),
).save("test_5.pdf", dpi=300)

# print(f_loki.info())
# print(np.diff(f_loki[-1].data[0][0].flatten()))
# print(np.diff(f_loki[-1].data)[0][0])

# data = f_loki[1].data[:, 32, 27]
# wavelength_arange = f_loki[-1].data[0][0].flatten() / (1 + 0.0099)
# from astropy.constants import c as light_speed
# hertz_conversion_factor = light_speed.to("micron/s").value / wavelength_arange
# data *= hertz_conversion_factor
# print(f_wicked[1].data[3, 32, 27] / data[3])
# print(data)

# print(f_loki[1].data[1944, 32, 27])
# raise
# gl.SmartFigure(elements=[
#     gl.Curve(np.arange(3814), f_wicked[1].data[:, 32, 27]),
#     gl.Curve(np.arange(2713), data * 5e2),
# ]).show()

In [ ]:
f_loki_all = fits.open("data/loki/output_NGC4696_G235H_F170LP_full_OQBr_tied/NGC4696_G235H_F170LP_full_OQBr_tied_full_model.fits")
f_loki_one = fits.open("data/loki/32_27/output_NGC4696_G235H_F170LP_32_27_QOBr_tied/NGC4696_G235H_F170LP_32_27_QOBr_tied_full_model.fits")
f_loki_all.info()

wv = f_loki_all[-1].data[0][0].flatten()

%matplotlib inline
gl.SmartFigure(elements=[
        gl.Curve(wv, f_loki_all[1].data[:, *FitsCoords(32, 27)], label="Spaxel (32, 27) from full cube"),
        gl.Curve(wv, f_loki_one[1].data[:, 0, 0], label="Spaxel (32, 27) from individual fit", line_width=1),
    ],
    x_lim=(2.65, 2.75),
    y_lim=(None, 0.7e-15),
    x_label="Wavelength (microns) (not corrected for redshift)",
    y_label="DATA from extension 1",
).set_visual_params(use_latex=True).show().save("full_vs_individual.png", dpi=300)


In [ ]:
arr = np.loadtxt("data/region_spectra/extracted_region_background.txt", skiprows=2)
gl.SmartFigure(elements=[gl.Curve(arr[:,0], arr[:,1])]).show()

In [ ]:
f_wicked = fits.open("data/wicked/f170lp_g235h-f170lp_s3d_choiclip1_2_wicked.fits")
c = Cube(f_wicked[1].data, f_wicked[1].header)
print(c.header["CDELT3"])
print(c.header["CRVAL3"], c.header["CRVAL3"] + c.header["CDELT3"] * (c.header["NAXIS3"] - 1))
print(c.header["CRVAL3"] / c.header["CDELT3"], (c.header["CRVAL3"] + c.header["CDELT3"] * (c.header["NAXIS3"] - 1)) / c.header["CDELT3"])